### Example
In this example, we write a function to calculate the mean of a list of numbers

In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from bs4 import BeautifulSoup as bs
from datetime import datetime
import json

In [81]:
def get_df(id, year):
    response = requests.get("https://www.nwac.us/data-portal/csv/q", {"datalogger_id": id, "year": year})
    return pd.read_csv(StringIO(response.text))

def get_soup(url):
    page = requests.get(url)
    return bs(page.text, 'lxml')

def get_portal_urls():
    soup = get_soup("https://www.nwac.us/data-portal/")
    div = soup.find("div", {"class":"entry-content"})
    elms = div.findAll("li")
    return [elm.find("a")["href"] for elm in elms]

def get_options(url):
    soup = get_soup("https://www.nwac.us" + url)
    optionElements = soup.findAll("select", {"name": "datalogger_id"})[0].findAll("option")
    return ([option.text for option in optionElements], [option["value"] for option in optionElements])

def get_data(year, portal):
    df_list = {}
    names, ids = get_options(f'/data-portal/location/{portal}/')
    for name, id in zip(names, ids):
        df_list[name] = get_df(id, year)
    return df_list

def get_avg_temps(year, portals):
    # What is the average temperature?
    temps = {}
    for portal in portals:
        data = get_data(year, portal)
        column = 'Temperature  (deg F) '
        avgs = []
        for key in data.keys():
            if column in data[key].columns:
                avgs.append(data[key].mean()[column])
        temps[portal] = np.mean(avgs)
    return temps

In [82]:
year = 2019
portals = [
    'hurricane-ridge',
    'mt-baker-ski-area',
#     'washington-pass',
#     'lake-wenatchee',
#     'stevens-pass',
#     'snoqualmie-pass',
#     'mt-rainier',
#     'white-pass-ski-area',
#     'mt-st-helens',
#     'mt-hood',
#     'crystal',
#     'chinook-pass'
]

In [84]:
get_avg_temps(year, portals)

{'hurricane-ridge': 38.98589634703201, 'mt-baker-ski-area': 39.56530690452371}

In [79]:
data = json.dumps({
    "year": 2019,
    "portals": portals
})
headers = {
    "content-type": "application/json",
    "sessiontoken": "",
}
response = requests.post("<iam_url>/executable/weather-demo/main/base/function/weather-data/run", data=data, headers=headers)

In [80]:
response.json()

{'result': {'crystal': 36.215856937024284}}